In [1]:
import glob
from asp_plot.icesat2 import Icesat2

In [2]:
dem_fn = glob.glob("/Users/ben/Dropbox/UW_Shean/WV/2022/WV03_20220417_1040010074793300_1040010075633C00/stereo*/*DEM_1m.tif")[0]

geojson_fn = "/Users/ben/Dropbox/UW_Shean/WV/2022/utqiagvik_wv_2022_sliderule_poly.geojson"

In [3]:
icesat2 = Icesat2(dem_fn=dem_fn, geojson_fn=geojson_fn)

In [4]:
atl06 = icesat2.pull_atl06_data()
atl06.head()

,spot,segment_id,n_fit_photons,dh_fit_dx,rms_misfit,h_mean,rgt,y_atc,w_surface_window_final,x_atc,region,h_sigma,gt,cycle,pflags,geometry,esa-worldcover-.time,esa-worldcover-.flags,esa-worldcover-.value,esa-worldcover-.file_id
time,,,,,,,,,,,,,,,,,,,,
2018-10-19 01:22:42.849838080,6,396539,5,0.004047,0.057618,0.712761,312,2075.261719,3.0,7947945.5,3,0.033456,10,1,0,POINT (-156.33771 71.24194),1.309046e+12,0,90.0,73014444032
2018-10-19 01:22:42.852651008,6,396540,10,0.009106,0.059513,0.823991,312,2075.311279,3.0,7947965.5,3,0.019844,10,1,0,POINT (-156.33778 71.24212),1.309046e+12,0,90.0,73014444032
2018-10-19 01:22:42.855464704,6,396541,15,-0.000860,0.093339,0.905593,312,2075.361572,3.0,7947985.5,3,0.024544,10,1,0,POINT (-156.33786 71.2423),1.309046e+12,0,90.0,73014444032
2018-10-19 01:22:42.858280448,6,396542,18,0.002180,0.091363,0.905599,312,2075.408936,3.0,7948005.5,3,0.021550,10,1,0,POINT (-156.33793 71.24247),1.309046e+12,0,90.0,73014444032
2018-10-19 01:22:42.861097984,6,396543,19,0.001642,0.104050,0.946535,312,2075.448486,3.0,7948025.5,3,0.024073,10,1,0,POINT (-156.338 71.24265),1.309046e+12,0,90.0,73014444032


In [5]:
atl06.shape

(325230, 20)